# Proposal 2

$$
\mathcal{L} = 
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    confusion_matrix_metrics,
    forgetting_metrics,
    timing_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    disk_usage_metrics,
    ram_usage_metrics,
    loss_metrics,
    class_accuracy_metrics,
    accuracy_matrix_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.training import CTSV1
from base_code.plugins import WeightStoragePlugin

# Constants

In [5]:
TORCH_DEVICE = "cuda" if cuda.is_available() else "cpu"
N_EXPERIENCES = 20
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"class_saturation/{N_EXPERIENCES}"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/proposal_2.pkl"

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

In [8]:
best_hyperparameters = {
    'lambda_l1': 0.1,
    'lambda_l2': 0.01,
    'eps': 1e-05
}

## Evaluation metrics definition

In [9]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    loss_metrics(minibatch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(minibatch=True, experience=True, stream=True),
    gpu_usage_metrics(experience=True, stream=True, gpu_id=TORCH_DEVICE),
    disk_usage_metrics(experience=True, stream=True),
    ram_usage_metrics(experience=True, stream=True),
    class_accuracy_metrics(experience=True, stream=True),
    accuracy_matrix_metrics(),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    confusion_matrix_metrics(stream=True, num_classes=scenario.n_classes),
    
    loggers=[InteractiveLogger()]
)

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/avalanche/evaluation/metrics/gpu_usage.py:56: UserWarning: Your system has no GPU!
  warnings.warn("Your system has no GPU!")
/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/avalanche/evaluation/metrics/gpu_usage.py:56: UserWarning: Your system has no GPU!
  warnings.warn("Your system has no GPU!")


## Plugin defitinitions

In [10]:
model_plugins = [WeightStoragePlugin()]

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [11]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = CTSV1(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    plugins=model_plugins,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
    **best_hyperparameters
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [12]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)

  0%|          | 0/20 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:14<00:00, 31.96it/s]
Epoch 0 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 95.5000
	Loss_MB/train_phase/train_stream/Task000 = 2.0938
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 14.6762
100%|██████████| 469/469 [00:16<00:00, 27.99it/s]
Epoch 1 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 39.3000
	Loss_MB/train_phase/train_stream/Task000 = 1.5859
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 16.7565
100%|██████████| 469/469 [00:23<00:00, 19.68it/s]
Epoch 2 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 13.3000
	Loss_MB/train_phase/train_stream/Task000 = 1.5958
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 23.8222
100%|██████████| 469/469 [00:23<00:00, 19.92it/s]
Epoch 3 ended.
	CPUUsage_MB/train_phase/train_stre

  5%|▌         | 1/20 [03:18<1:02:52, 198.55s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 44.0044
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x1380f3df0>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9053, 0.1338, 0.1285, 0.1664, 0.1305, 0.1065, 0.1229, 0.0937, 0.1219,
         0.1185, 0.0872, 0.0911, 0.1063, 0.1139, 0.1063, 0.1016, 0.1068, 0.1176,
         0.0719, 0.1232],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

 10%|█         | 2/20 [06:29<58:12, 194.02s/it]  

...

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 48.5055
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x13cb6d040>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9053, 0.1338, 0.1285, 0.1664, 0.1305, 0.1065, 0.1229, 0.0937, 0.1219,
         0.1185, 0.0872, 0.0911, 0.1063, 0.1139, 0.1063, 0.1016, 0.1068, 0.1176,
         0.0719, 0.1232],
        [0.8813, 0.9241, 0.1448, 0.1317, 0.1273, 0.0863, 0.0939, 0.0825, 0.1178,
         0.1738, 0.1086, 0.1041, 0.1245, 0.0822, 0.1300, 0.0931, 0.0944, 0.1021,
         0.0987, 0.1171],
        [0.8565, 0.8927, 0.9330, 0.1111, 0.0982, 0.1049, 0.0756, 0.1059, 0.1063,
         0.1373, 0.0897, 0.0918, 0.1196, 0.0661, 0.1538, 0.0711, 0.0880, 0.1225,
         0.1115, 0.0969],
        [0.7975, 0.8555, 0.9142, 0.9356, 0.0681, 0.0756, 0.0899, 0.1044, 0.0942,
         0.1222, 0.0880, 0.1235, 0.1569, 0.0774, 0.18

 35%|███▌      | 7/20 [25:53<50:54, 234.96s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 42.4481
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x13da58730>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9053, 0.1338, 0.1285, 0.1664, 0.1305, 0.1065, 0.1229, 0.0937, 0.1219,
         0.1185, 0.0872, 0.0911, 0.1063, 0.1139, 0.1063, 0.1016, 0.1068, 0.1176,
         0.0719, 0.1232],
        [0.8813, 0.9241, 0.1448, 0.1317, 0.1273, 0.0863, 0.0939, 0.0825, 0.1178,
         0.1738, 0.1086, 0.1041, 0.1245, 0.0822, 0.1300, 0.0931, 0.0944, 0.1021,
         0.0987, 0.1171],
        [0.8565, 0.8927, 0.9330, 0.1111, 0.0982, 0.1049, 0.0756, 0.1059, 0.1063,
         0.1373, 0.0897, 0.0918, 0.1196, 0.0661, 0.1538, 0.0711, 0.0880, 0.1225,
         0.1115, 0.0969],
        [0.7975, 0.8555, 0.9142, 0.9356, 0.0681, 0.0756, 0.0899, 0.1044, 0.0942,
         0.1222, 0.0880, 0.1235, 0.1569, 0.0774, 0.18

 40%|████      | 8/20 [29:22<45:21, 226.79s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 50.3540
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x13e280b20>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9053, 0.1338, 0.1285, 0.1664, 0.1305, 0.1065, 0.1229, 0.0937, 0.1219,
         0.1185, 0.0872, 0.0911, 0.1063, 0.1139, 0.1063, 0.1016, 0.1068, 0.1176,
         0.0719, 0.1232],
        [0.8813, 0.9241, 0.1448, 0.1317, 0.1273, 0.0863, 0.0939, 0.0825, 0.1178,
         0.1738, 0.1086, 0.1041, 0.1245, 0.0822, 0.1300, 0.0931, 0.0944, 0.1021,
         0.0987, 0.1171],
        [0.8565, 0.8927, 0.9330, 0.1111, 0.0982, 0.1049, 0.0756, 0.1059, 0.1063,
         0.1373, 0.0897, 0.0918, 0.1196, 0.0661, 0.1538, 0.0711, 0.0880, 0.1225,
         0.1115, 0.0969],
        [0.7975, 0.8555, 0.9142, 0.9356, 0.0681, 0.0756, 0.0899, 0.1044, 0.0942,
         0.1222, 0.0880, 0.1235, 0.1569, 0.0774, 0.18

# Get metrics

In [ ]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()
class_accuracies: dict[int, list[float]] = dict()
losses: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    class_accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/{i}"
    ][1]
    losses[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Loss_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

In [ ]:
train_cpu_usage = eval_plugin.get_all_metrics()["CPUUsage_MB/train_phase/train_stream/Task000"][1]
train_time_epoch = eval_plugin.get_all_metrics()["Time_Epoch/train_phase/train_stream/Task000"][1]
train_running_time = eval_plugin.get_all_metrics()["RunningTime_Epoch/train_phase/train_stream/Task000"][1]
train_loss_epoch = eval_plugin.get_all_metrics()["Loss_MB/train_phase/train_stream/Task000"][1]

In [ ]:
all_loss_components = strategy.get_store_loss()

In [ ]:
weights = model_plugins[0].weights

# Store metrics

In [ ]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
    "class_accuracies": class_accuracies,
    "losses": losses,
    "train_cpu_usage": train_cpu_usage,
    "train_time_epoch": train_time_epoch,
    "train_running_time": train_running_time,
    "train_loss_epoch": train_loss_epoch,
    "all_loss_components": all_loss_components,
    "weights": weights
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))